In [ ]:
# 为了说明使用对象的隐喻不需要特定的编程语言
# 这章用函数和字典实现类和对象
# 在没有内置对象系统的编程语言中，也是可以面向对象的

In [ ]:
# 定义实例对象
def make_instance(cls):  # 以'类'(这里'类'是我们要实现的东西,这里实际是一个调度字典)作为形式参数
    '''返回一个新的实例对象,是一个调度字典'''
    def get_value(name):
        if name in attributes:  # 属性存储在一个名为attributes的局部字典中
            return attributes[name]
        else:
            value = cls['get'](name)   # 属性name如果没有在实例的属性(局部的attributes字典)中,则在类中查找
            return bind_method(value,instance)
    def set_value(name,value):
        '''与python对象系统中的属性赋值操作对应'''
        attributes[name] = value
    attributes = {}
    instance = {'get':get_value,'set':set_value}
    return instance

def bind_method(value,instance):
    '''如果value是可调用的函数,则返回绑定了对象的方法;如果value是值,则返回其值'''
    if callable(value):
        def method(*args):
            return value(instance,*args)
        return method
    else:
        return value

# 定义类
def make_class(attributes,base_class=None):
    '''返回一个新的类,是一个调度字典'''
    def get_value(name):
        if name in attributes:
            return attributes[name]
        elif base_class is not None:  # 如果当前类中没有名为name的属性,则到其基类(父类)中查找
            return base_class['get'](name)
    def set_value(name,value):
        attributes[name] = value
    def new(*args):    # 新建对象
        return init_instance(cls,*args)
    cls = {'get':get_value,'set':set_value,'new':new}
    return cls

def init_instance(cls,*args):
    '''创建一个新的空实例对象,并调用'__init__'为其填充创建的类中所定义的初始化属性'''
    instance = make_instance(cls)
    init = cls['get']('__init__')
    if init:     # ?
        init(instance,*args)
    return instance


In [17]:
# 使用上面实现的对象系统,以2.5的账户为例
# 创建类
def make_account_class():
    '''创建一个Account类,有deposit和withdraw方法'''
    interest = 0.02
    def __init__(self,account_holder):
        self['set']('holder',account_holder)
        self['set']('balance',0)
    def deposit(self,amount):
        new_balance = self['get']('balance')+amount
        self['set']('balance',new_balance)
        return self['get']('balance')
    def withdraw(self,amount):
        balance = self['get']('balance')
        if amount > balance:
            return 'Insufficient funds'
        self['set']('balance',balance - amount)
        return self['get']('balance')
    # print(locals())
    return make_class(locals())

In [15]:
make_account_class()

{'interest': 0.02, '__init__': <function make_account_class.<locals>.__init__ at 0x0000019DE36FFE20>, 'deposit': <function make_account_class.<locals>.deposit at 0x0000019DE36FFD80>, 'withdraw': <function make_account_class.<locals>.withdraw at 0x0000019DE36FFEC0>}


{'get': <function __main__.make_class.<locals>.get_value(name)>,
 'set': <function __main__.make_class.<locals>.set_value(name, value)>,
 'new': <function __main__.make_class.<locals>.new(*args)>}

In [18]:
Account = make_account_class()

In [19]:
kirk_account = Account['new']('kirk')

In [20]:
print(kirk_account['get']('holder'))
print(kirk_account['get']('interest'))
print(kirk_account['get']('deposit')(20))
print(kirk_account['get']('withdraw')(5))

kirk
0.02
20
15


In [21]:
kirk_account['set']('interest',0.04)
Account['get']('interest')

0.02

In [22]:
# 创建Account的子类
def make_checking_account_class():
    interest = 0.01
    withdraw_fee = 1  # withdraw_charge
    def withdraw(self,amount):
        fee = self['get']('withdraw_fee')
        return Account['get']('withdraw')(self,amount+fee)
    return make_class(locals(),Account)

In [23]:
CheckingAccount = make_checking_account_class()
jack_acct = CheckingAccount['new']('Spock')

In [24]:
print(jack_acct['get']('interest'))
print(jack_acct['get']('deposit')(20))
print(jack_acct['get']('withdraw')(5))

0.01
20
14
